### Import Libraries

In [15]:
import importlib
import os
import json
import sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from models import clap, beats
from actions import helpers

importlib.reload(clap)
importlib.reload(beats)
importlib.reload(helpers)
from models.clap import CLAPModel
from models.beats import BeatsModel

In [7]:
CLAP_OUTPUT_JSON_PATH = "../../data/results/clap_results.json"
BEATS_OUTPUT_JSON_PATH = "../../data/results/beats_results.json"

### Get video paths and candidate labels

In [3]:
xd_violence = os.path.join(os.getenv("XD_VIOLENCE_PATH"), "abuse2.mp4")

In [4]:
audio_labels, audioset_labels = helpers.extract_audio_data(os.getenv("LABELS_PATH"))
primary_labels = list(audio_labels.keys())

In [ ]:
# 3. Load video paths
video_urls_path = "../../data/metadata/video_links_labels.csv"
video_df = pd.read_csv(video_urls_path)

video_folder = "../../data/videos"
video_paths = [
    os.path.join(video_folder, os.path.basename(url))
    for url in video_df["download_url"].tolist()
    if os.path.exists(os.path.join(video_folder, os.path.basename(url)))
]

### Run Clap

In [8]:
#  Initialize CLAP model
clap_model = CLAPModel(
    model_name="laion/clap-htsat-fused",
    sr=48000,  # CLAP expects 48kHz audio
    window_s=10.0,  # 10-second windows
    hop_s=10.0,  # Non-overlapping chunks
)

Device set to use cpu


Loaded CLAP model: ClapFeatureExtractor {
  "chunk_length_s": 10,
  "feature_extractor_type": "ClapFeatureExtractor",
  "feature_size": 64,
  "fft_window_size": 1024,
  "frequency_max": 14000,
  "frequency_min": 50,
  "hop_length": 480,
  "max_length_s": 10,
  "n_fft": 1024,
  "nb_frequency_bins": 513,
  "nb_max_frames": 1000,
  "nb_max_samples": 480000,
  "padding": "repeatpad",
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "ClapProcessor",
  "return_attention_mask": false,
  "sampling_rate": 48000,
  "top_db": null,
  "truncation": "fusion"
}



In [10]:
await clap_model.process_all_videos(
    video_paths=[xd_violence],
    labels=primary_labels,
    output_json=CLAP_OUTPUT_JSON_PATH,
    overwrite=False,
    top_k=3,
)

c:\Users\annak\Documents\Multimodal-Techniques-for-Video-Based-Violence-Detection\models\clap.py:70: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(audio_path, sr=self.sr, mono=True)
c:\Users\annak\Documents\Multimodal-Techniques-for-Video-Based-Violence-Detection\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processed 1 videos


In [11]:
await clap_model.process_all_videos(
    video_paths=[xd_violence],
    labels=audioset_labels,
    output_json=CLAP_OUTPUT_JSON_PATH,
    overwrite=False,
    top_k=3,
)

c:\Users\annak\Documents\Multimodal-Techniques-for-Video-Based-Violence-Detection\models\clap.py:70: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(audio_path, sr=self.sr, mono=True)
c:\Users\annak\Documents\Multimodal-Techniques-for-Video-Based-Violence-Detection\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processed 1 videos


### Run BEATS

In [16]:
# Initialize BEATs model
beats_model = BeatsModel(
    checkpoint_path=os.getenv("BEATS_MODEL_PATH"),
    human_labels_path=os.getenv("ONTOLOGY_JSON_PATH"),
    sr=16000,  # BEATs expects 16kHz audio
    win_ms=2000,  # 2-second windows
    hop_ms=2000,  # Non-overlapping chunks
)

c:\Users\annak\Documents\Multimodal-Techniques-for-Video-Based-Violence-Detection\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded BEATs model


In [17]:
await beats_model.process_all_videos(
    video_paths=[xd_violence],
    output_json=BEATS_OUTPUT_JSON_PATH,
    overwrite=False,
    top_k=3,
)

Processed 1 videos
